# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.preprocessing import StandardScaler
clf = StandardScaler()
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


StandardScaler()

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lOutputs;
	}

	tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {

		tTable lTable;

		lTable["Feature_0"] = { ( ( Feature_0 - 5.843333333333334 ) / 0.8253012917851409 ) };
		lTable["Feature_1"] = { ( ( Feature_1 - 3.0573333333333337 ) / 0.4344109677354946 ) };
		lTable["Feature_2"] = { ( ( Feature_2 - 3.7580000000000005 ) / 1.759404065775303 ) };
		lTable["Feature_3"] = { ( ( Feature_3 - 1.1993333333333336 ) / 0.7596926279021594 ) };

		return lTable;
	}

	tTable compute_model_outputs_from_table( tTable const & iTable) {
		tTable lTable = compute_features(iTable.at("Feature_0")[0], iTable.at("Feature_1")[0],

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140263582503120.exe /tmp/sklearn2sql_cpp_140263582503120.cpp'
idx,Feature_0,Feature_1,Feature_2,Feature_3
0,-0.90068117029781,1.01900435197161,-1.34022652662276,-1.31544429500774
1,-1.14301691118511,-0.13197947932162,-1.34022652662276,-1.31544429500774
2,-1.38535265207241,0.32841405319567,-1.39706395353637,-1.31544429500774
3,-1.50652052251607,0.09821728693702,-1.28338909970916,-1.31544429500774
4,-1.02184904074146,1.24920111823025,-1.34022652662276,-1.31544429500774
5,-0.53717755896685,1.93979141700619,-1.16971424588195,-1.05217992642714
6,-1.50652052251607,0.78880758571296,-1.34022652662276,-1.18381211071744
7,-1.02184904074146,0.78880758571296,-1.28338909970916,-1.31544429500774
8,-1.74885626340337,-0.36217624558027,-1.34022652662276,-1.31544429500774
9,-1.14301691118511,0.09821728693702,-1.28338909970916,-1.44707647929804
10,-0.53717755896685,1.47939788448890,-1.28338909970916,-1.31544429500774
11,-1.26

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,-0.052506,-0.592373,0.762758,1.580464
74,74,0.674501,-0.362176,0.308059,0.132510
9,9,-1.143017,0.098217,-1.283389,-1.447076
88,88,-0.294842,-0.131979,0.194384,0.132510
25,25,-1.021849,-0.131979,-1.226552,-1.315444
5,5,-0.537178,1.939791,-1.169714,-1.052180
48,48,-0.658345,1.479398,-1.283389,-1.315444
117,117,2.249683,1.709595,1.672157,1.317199
83,83,0.189830,-0.822570,0.762758,0.527406
105,105,2.128516,-0.131979,1.615320,1.185567


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_transform = pd.DataFrame(clf.transform(X), columns=cpp_output.columns[1:]);
skl_output = pd.concat([skl_output_key, skl_output_transform] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,-0.052506,-0.592373,0.762758,1.580464
74,74,0.674501,-0.362176,0.308059,0.132510
9,9,-1.143017,0.098217,-1.283389,-1.447076
88,88,-0.294842,-0.131979,0.194384,0.132510
25,25,-1.021849,-0.131979,-1.226552,-1.315444
5,5,-0.537178,1.939791,-1.169714,-1.052180
48,48,-0.658345,1.479398,-1.283389,-1.315444
117,117,2.249683,1.709595,1.672157,1.317199
83,83,0.189830,-0.822570,0.762758,0.527406
105,105,2.128516,-0.131979,1.615320,1.185567


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Feature_0_skl,Feature_1_skl,Feature_2_skl,Feature_3_skl,idx_cpp,Feature_0_cpp,Feature_1_cpp,Feature_2_cpp,Feature_3_cpp
114,114,-0.052506,-0.592373,0.762758,1.580464,114,-0.052506,-0.592373,0.762758,1.580464
74,74,0.674501,-0.362176,0.308059,0.132510,74,0.674501,-0.362176,0.308059,0.132510
9,9,-1.143017,0.098217,-1.283389,-1.447076,9,-1.143017,0.098217,-1.283389,-1.447076
88,88,-0.294842,-0.131979,0.194384,0.132510,88,-0.294842,-0.131979,0.194384,0.132510
25,25,-1.021849,-0.131979,-1.226552,-1.315444,25,-1.021849,-0.131979,-1.226552,-1.315444
5,5,-0.537178,1.939791,-1.169714,-1.052180,5,-0.537178,1.939791,-1.169714,-1.052180
48,48,-0.658345,1.479398,-1.283389,-1.315444,48,-0.658345,1.479398,-1.283389,-1.315444
117,117,2.249683,1.709595,1.672157,1.317199,117,2.249683,1.709595,1.672157,1.317199
83,83,0.189830,-0.822570,0.762758,0.527406,83,0.189830,-0.822570,0.762758,0.527406
105,105,2.128516,-0.131979,1.615320,1.185567,105,2.128516,-0.131979,1.615320,1.185567


In [14]:
for col in cpp_output.columns:
    lDiff = cpp_skl_join[col + "_skl"] - cpp_skl_join[col + "_cpp"]
    print(lDiff.describe())

count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    1.500000e+02
mean    -2.066865e-16
std      2.732826e-15
min     -4.662937e-15
25%     -2.942091e-15
50%     -4.163336e-16
75%      2.373102e-15
max      4.884981e-15
dtype: float64
count    1.500000e+02
mean    -7.393160e-16
std      2.972491e-15
min     -4.690692e-15
25%     -3.108624e-15
50%     -1.221245e-15
75%      1.817990e-15
max      4.884981e-15
dtype: float64
count    1.500000e+02
mean     7.017535e-17
std      3.067320e-15
min     -4.940492e-15
25%     -2.442491e-15
50%      6.661338e-16
75%      2.886580e-15
max      5.107026e-15
dtype: float64
count    1.500000e+02
mean    -2.413289e-16
std      2.261244e-15
min     -4.884981e-15
25%     -1.675960e-15
50%     -2.220446e-16
75%      9.992007e-16
max      4.329870e-15
dtype: float64
